In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
import logging
import random
from random import choice

import numpy as np
import tensorflow as tf

import utils
from model import Model

from flags import parse_args
FLAGS, unparsed = parse_args()

FLAGS.text = 'QuanSongCi.txt'
FLAGS.dictionary = 'embedding.npy'
FLAGS.reverse_dictionary = 'reversed_embedding.npy'
FLAGS.learning_rate = 0.01
FLAGS.batch_size = 16
FLAGS.num_steps = 10
FLAGS.output_dir = './'
verbose = True

logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s', level=logging.DEBUG)


with open(FLAGS.dictionary, encoding='utf-8') as inf:
    dictionary = json.load(inf, encoding='utf-8')

with open(FLAGS.reverse_dictionary, encoding='utf-8') as inf:
    reverse_dictionary = json.load(inf, encoding='utf-8')


reverse_list = [reverse_dictionary[str(i)]
                for i in range(len(reverse_dictionary))]
titles = ['江神子', '蝶恋花', '渔家傲']


model = Model(learning_rate=FLAGS.learning_rate, batch_size=1, num_steps=1)
model.build()

with tf.Session() as sess:
    summary_string_writer = tf.summary.FileWriter(FLAGS.output_dir, sess.graph)

    saver = tf.train.Saver(max_to_keep=5)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    logging.debug('Initialized')

    try:
        checkpoint_path = tf.train.latest_checkpoint(FLAGS.output_dir)
        saver.restore(sess, checkpoint_path)
        logging.debug('restore from [{0}]'.format(checkpoint_path))

    except Exception as E:
        logging.debug('no check point found....')
        print('E=',E)
        exit(0)

    for title in titles:
        state = sess.run(model.state_tensor)
        # feed title
        for head in title:
            input = utils.index_data(np.array([[head]]), dictionary)

            feed_dict = {model.X: input,
                         model.state_tensor: state,
                         model.keep_prob: 1.0}

            pred, state = sess.run(
                [model.predictions, model.outputs_state_tensor], feed_dict=feed_dict)

        sentence = title
        word_index = pred[0].argsort()[-1]

        # generate sample
        for i in range(64):
            feed_dict = {model.X: [[word_index]],
                         model.state_tensor: state,
                         model.keep_prob: 1.0}

            pred, state = sess.run(
                [model.predictions, model.outputs_state_tensor], feed_dict=feed_dict)

            word_index = pred[0].argsort()[-1]
            
            tmp = pred[0].argsort()[-8:]
#             print('tmp=', tmp)
            word_index = choice(tmp)
            if 2 == word_index:
                word_index = pred[0].argsort()[-2]
#             for i in pred[0].argsort()[-11:]:
#                 print('reverse_list[i]=', reverse_list[i])
            
            word = np.take(reverse_list, word_index)
            sentence = sentence + word


        logging.debug('==============[{0}]=============='.format(title))
        logging.debug(sentence)


2019-02-18 10:47:47,935 - DEBUG - <ipython-input-1-87f9b1111895>:52 - Initialized


INFO:tensorflow:Restoring parameters from ./model.ckpt-10810


2019-02-18 10:47:47,941 - INFO - tf_logging.py:115 - Restoring parameters from ./model.ckpt-10810
2019-02-18 10:47:47,965 - DEBUG - <ipython-input-1-87f9b1111895>:57 - restore from [./model.ckpt-10810]
2019-02-18 10:47:48,169 - DEBUG - <ipython-input-1-87f9b1111895>:103 - ==============[江神子]==============
2019-02-18 10:47:48,170 - DEBUG - <ipython-input-1-87f9b1111895>:104 - 江神子天月。一日清风无见去无。不似西山月雨处去人情处。且有西湖。莫用，今人，更人人。且不见。不用人、无是不来时去、何似不人来。更今事不
2019-02-18 10:47:48,337 - DEBUG - <ipython-input-1-87f9b1111895>:103 - ==============[蝶恋花]==============
2019-02-18 10:47:48,338 - DEBUG - <ipython-input-1-87f9b1111895>:104 - 蝶恋花人日春来何处处来风雨。且有无间不是春，无事。更是一声无去在来何许无人处。一里西花、不用无心处，只知何是是归情路。只教无处人年去处
2019-02-18 10:47:48,504 - DEBUG - <ipython-input-1-87f9b1111895>:103 - ==============[渔家傲]==============
2019-02-18 10:47:48,505 - DEBUG - <ipython-input-1-87f9b1111895>:104 - 渔家傲更得无人一笑处、何是。且知、一笑，人日春。且不为春处路路春风雨。莫是春情，人时、不似、不人无在去、一里一枝，不似东楼。只有清年一
